In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
#from visualization import visualize, display, sequence_identity
from rdkit.Chem import AllChem
from scipy.stats import spearmanr, ttest_ind
import os

#supress warnings
import warnings
warnings.filterwarnings("ignore")

### Task 2: process reaction similarities into a dataframe (only necessary for CLIPZyme and CARE)

Technically don't need Random to be generated here since it is replaced later anyways

In [5]:
for baseline in  ['Similarity', 'CLIPZyme', 'CREEP', 'CREEP_text']:
    for split in ['easy', 'medium', 'hard']:

        if baseline == 'Similarity':
            reaction_similarities = np.load('/disk1/jyang4/repos/CARE/task2_baselines/Similarity/output/{}_split/retrieval_results/{}_reaction_test_reaction2reaction_retrieval_similarities.npy'.format(split, split))
        elif 'CREEP' in baseline:
            reaction_similarities = np.load('/disk1/jyang4/repos/CARE/task2_baselines/CREEP/output/{}_split/retrieval_results/{}_reaction_test_reaction2protein_retrieval_similarities.npy'.format(split, split))
            if 'text' in baseline:
                text_similarities = np.load('/disk1/jyang4/repos/CARE/task2_baselines/CREEP/output/{}_split/retrieval_results/{}_reaction_test_text2protein_retrieval_similarities.npy'.format(split, split))
        elif baseline == 'CLIPZyme':
            reaction_similarities = np.load('/disk1/jyang4/repos/CARE/task2_baselines/CLIPZyme/output/{}_split/retrieval_results/{}_reaction_test_reaction2protein_retrieval_similarities.npy'.format(split, split))
        

        query_df = pd.read_csv('../splits/task2/{}_reaction_test.csv'.format(split))
        query_EC_list = query_df['EC number'].values
        num_cols = len(query_df.columns)

        #add len(reaction_similarities) as empty columns to the query_df
        for i in range(reaction_similarities.shape[1]):
            query_df[i] = np.nan
        
        reference_EC_list = np.loadtxt('../processed_data/EC_list.txt', dtype=str)

        #get the number of columns in the query_df
        for i, query_EC in enumerate(query_EC_list):
            reaction_similarity = reaction_similarities[i]
            
            df = pd.DataFrame({'EC': reference_EC_list, 'reaction similarity': reaction_similarity})
            df['reaction rank'] = df['reaction similarity'].rank(ascending=False)
            if 'text' in baseline:
                text_similarity = text_similarities[i]
                df['text similarity'] = text_similarity
                df['text rank'] = df['text similarity'].rank(ascending=False)
                df['overall rank'] = (df['reaction rank'] + df['text rank'])/2
            else:
                df['overall rank'] = df['reaction rank']
                
            df.sort_values('overall rank', ascending=True, inplace=True) #need to check if this order is correct
            #append df['EC'] to the query_df in the numerical columns

            ### use this if similarity is based on euclidean distance rather than dot product ###
            #df.sort_values('overall rank', ascending=False, inplace=True)
            if baseline == 'Random':
                np.random.seed(42)
                query_df.iloc[i, num_cols:] = df['EC'].sample(frac=1).values
            else:
                query_df.iloc[i, num_cols:] = df['EC'].values
        
        #ensure the directory exists
        
        if not os.path.exists('results_summary/{}'.format(baseline)):
            os.makedirs('results_summary/{}'.format(baseline))

        query_df.to_csv('results_summary/{}/{}_reaction_test_results_df.csv'.format(baseline,split), index=False)